# PyMentat を用いた有限要素モデルの作成
Mentat用のmfdファイルを直接出力

In [11]:
# PyMentatモジュールのインポート
# from py_mentat import *
import math

In [12]:
# パラメータの設定
z = [0, 3700, 3700*2, 3700*3, 3700*4, 3700*5, 3700*6, 3700*7, 3700*8, 3700*9, 3700*10] # 基準点(mm)
t = [350, 330, 310, 290, 270, 250, 230, 210, 190, 170, 150] # 板厚(mm)
r = [1308, 1249, 1226, 1203, 1174, 1130, 1097, 1061, 1016, 975, 953] # 半径(mm)
n = [3, 100, 40] # (r, theta, z)方向の区間ごと分割数
r_method = 0 # 0:外接円、1:内接円、2:(外接円+内接円)/2

In [13]:
# ループのため利用するデータを区間ごとにまとめておく
z1 = z[0:len(z)-1]
z2 = z[1:len(z)]
t1 = t[0:len(t)-1]
t2 = t[1:len(t)]
r1 = r[0:len(r)-1]
r2 = r[1:len(r)]

data = []
for i in range(0, len(z1)):
    data.append([z1[i], z2[i], t1[i], t2[i], r1[i], r2[i]])

In [14]:
# 座標値の作成
nodes = []
# 区間ごとにループを実行(最後の区間は別に)
for (z1, z2, t1, t2, r1, r2) in data[:len(data)-1]:
    # z座標を分割
    z_list = [(z2 - z1) / n[2] * i + z1 for i in range(n[2])]
    
    # z座標におけるr, tを計算
    t_list = [(t2 - t1) / n[2] * i + t1 for i in range(n[2])]
    r_list = [(r2 - r1) / n[2] * i + r1 for i in range(n[2])]
    
    for k in range(len(z_list)):
        # z
        zz = z_list[k]
        
        # r方向の分割値
        R = r_list[k] / math.cos(math.pi / n[1]) # rr を内接円とした場合の多角形の頂点までの距離
        if r_method == 0: # 外接円
            rr = r_list[k]
        elif r_method == 1: # 内接円
            rr = R
        elif r_method == 2: # (外接円+内接円)/2
            rr = (r_list[k] + R) / 2

        tt = t_list[k]
        dr_list = [(rr - tt) + tt / n[0] * i for i in range(n[0]+1)]
        
        # theta方向の分割値(radian)
        rad_list = [2 * math.pi / n[1] * i for i in range(n[1])]
        
        for dr in dr_list:
            for rad in rad_list:
                x = dr * math.cos(rad)
                y = dr * math.sin(rad)
                nodes.append([x, y, zz])
        
# 最後の区間
(z1, z2, t1, t2, r1, r2) = data[-1]

# z座標を分割
z_list = [(z2 - z1) / n[2] * i + z1 for i in range(n[2]+1)]
    
# z座標におけるr, tを計算
t_list = [(t2 - t1) / n[2] * i + t1 for i in range(n[2]+1)]
r_list = [(r2 - r1) / n[2] * i + r1 for i in range(n[2]+1)]
    
for k in range(len(z_list)):
    # z
    zz = z_list[k]
        
    # r方向の分割値
    R = r_list[k] / math.cos(math.pi / n[1]) # rr を内接円とした場合の多角形の頂点までの距離
    if r_method == 0: # 外接円
        rr = r_list[k]
    elif r_method == 1: # 内接円
        rr = R
    elif r_method == 2: # (外接円+内接円)/2
        rr = (r_list[k] + R) / 2
   
    tt = t_list[k]
    dr_list = [(rr - tt) + tt / n[0] * i for i in range(n[0]+1)]
        
    # theta方向の分割値(radian)
    rad_list = [2 * math.pi / n[1] * i for i in range(n[1])]
        
    for dr in dr_list:
        for rad in rad_list:
            x = dr * math.cos(rad)
            y = dr * math.sin(rad)
            nodes.append([x, y, zz])

In [15]:
# 要素の作成
n_blocks = len(z) - 1

num_z = n[2]
num_r = n[0]
num_theta = n[1]
num_layer = (num_r + 1) * num_theta # 各層ごとの節点数

elems = []
for k in range(num_z * n_blocks): # z方向
    for j in range(num_r): # r方向
        for i in range(num_theta): # theta方向
            if i == num_theta - 1: # 1周回ったら補正
                o = (num_layer * k) + num_theta * j + i + 1 # 基準点
                e = [o, o+num_theta, o+1, o-num_theta+1, o+num_layer, o+num_layer+num_theta, o+num_layer+1, o+num_layer-num_theta+1]
            else:
                o = (num_layer * k) + num_theta * j + i + 1 # 基準点
                e = [o, o+num_theta, o+num_theta+1, o+1, o+num_layer, o+num_layer+num_theta, o+num_layer+num_theta+1, o+num_layer+1]
            elems.append(e)

In [16]:
# 要素セットの作成
num_elem_par_block = num_theta * num_r * num_z
elem_sets = []
for j in range(n_blocks):
    elem_set = []
    for i in range(num_elem_par_block):
        elem_set.append(num_elem_par_block * j + i)
    elem_sets.append(elem_set)

In [17]:
# 出力ファイル名
output_filename = 'chimney.mfd'

In [18]:
# MFDファイルへの出力
with open(output_filename, mode='w') as f:
    # MFDファイルヘッダーの出力
    f.write('Version : Marc Mentat 2010.2.0 (64bit)\n')
    f.write('=beg=    1 (magic)\n')
    f.write('{:>20d}\n'.format(1234))
    f.write('=end=\n')
    f.write('=beg=    2 (entities)\n')
    f.write('{:>20d}\n'.format(len(nodes) + len(elems) + len(elem_sets)))
    f.write('=end=\n')
    f.write('=beg=    3 (description)\n\n')
    f.write('=end=\n')

    # 節点情報の出力
    f.write('=beg=  102 (nodes)\n')
    for i, node in enumerate(nodes):
        f.write('{:>20d} {: 13.12e} {: 13.12e} {: 13.12e}\n'.format(i+1, node[0], node[1], node[2]))
        f.write('{:>20d}{:>20d}\n'.format(0, 0))
    f.write('=end=\n')

    # 要素情報の出力
    f.write('=beg=  205 (elements)\n')
    for i, elem in enumerate(elems):
        f.write('{:>20d}{:>20d}{:>20d}{:>20d}\n'.format(i+1, 8, 0, 8))
        f.write('{:>20d}{:>20d}{:>20d}{:>20d}\n'.format(elem[0], elem[1], elem[2], elem[3]))
        f.write('{:>20d}{:>20d}{:>20d}{:>20d}\n'.format(elem[4], elem[5], elem[6], elem[7]))
        f.write('{:>20d}{:>20d}{:>20d}{:>20d}\n'.format(0, 0, 0, 0))
        f.write('{:>20d}{:>20d}{:>20d}{:>20d}\n'.format(0, 0, 0, 0))
    f.write('=end=\n')
    
    # セット情報の出力
    f.write('=beg= 1203 (sets)\n')
    for j, elem_set in enumerate(elem_sets):
        f.write('set{:02d}\n'.format(j+1))
        f.write('{:>20d}\n'.format(0))
        f.write('{:>20d}\n'.format(0))
        f.write('{:>20d}{:>20d}{:>20d}\n'.format(j+1, 1, 1))
        f.write('{:>20d}\n'.format(len(elem_set)))
        for i, e_no in enumerate(elem_set):
            q, mod = divmod((i + 1), 4)
            f.write('{:>20d}'.format(e_no+1))
            if mod == 0:
                f.write('\n')
        if mod != 0:
            f.write('\n')
        for i, e_no in enumerate(elem_set):
            q, mod = divmod((i+1), 4)
            f.write('{:>20d}'.format(0))
            if mod == 0:
                f.write('\n')
        if mod != 0:
            f.write('\n')
        for i, e_no in enumerate(elem_set):
            q, mod = divmod((i + 1), 4)
            f.write('{:>20d}'.format(0))
            if mod == 0:
                f.write('\n')
        if mod != 0:
            f.write('\n')
    f.write('=end=\n')